# Data wrangling approach

Things to verify:
1. Are the 4 features(counts) useful as predictors or is it okay to throw them away?  

# Modeling Approach

1. Generate sliding windowed data set as before
2. Train test split as before
3. Unroll matrix at country of origin dimension and treat each sample differently
4. Standardize and log transform on just 1 feature
5. Model the spatio-temporal data on different baselines (minus graph data)

# Visualization approach

1. Model the diffusion on an actual world map?

In [22]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
import h5py
import gzip
import json
import os
import tqdm

import warnings

warnings.filterwarnings("ignore")

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.WARN)

## Load data

In [5]:
incidences = None
with h5py.File("data/EpiGCN/processing/incidences.hdf5", "r") as f:
    incidences = f['incidences'][:]

In [6]:
incidences.shape

(3181, 232, 366, 4)

### Normalize by population to get fraction of population infected

In [7]:
import json
countries_metadata = json.load(open('data/countries_metadata.json'))

In [8]:
with gzip.open('data/output_0000.h5.gz') as fp:
    f = h5py.File(fp,'r')
    compartments = f['dset'].attrs['Columns']
    country_ids = f['dset'].attrs['SelectedCountries']
    prevalence = np.array(f['dset/countries/cumulative'], dtype=np.int64)
    incidence = np.array(f['dset/countries/transitions'], dtype=np.int64)

In [9]:
country_ids

array([ 71, 140, 149, 222, 103, 183, 199,  53, 113,  29, 221, 207,  47,
        28, 213,  76, 157, 109, 181, 154, 164, 178,  50,  79, 108, 185,
         7,  31, 128,  91,  75, 227,  70,  80,  81,  12, 179, 176, 172,
       210,  94,  67, 200, 129, 218, 215, 110, 188,   2,  82, 187, 150,
        61, 177, 119,  16, 180,  95, 208, 147,  54,  86,  15,  88,  60,
        90, 158, 230,  58,   8, 144,  89, 202, 130, 104, 138,  10,  13,
       161,  35, 193, 111, 229,  72,  48, 153,   5,  84, 117,  22, 191,
       112,  73, 120, 214, 107,  62, 116,  85,   0, 206, 115,  34,  25,
        45, 160, 205,  49,  78, 159,  63, 106, 201, 125, 216, 122, 134,
         3,  52, 223,  43, 123,  41,  51, 135, 169, 101, 197,  39, 139,
         1,  27,  55,  83,  23,   6,  65, 190,  74, 231, 145, 196, 211,
       141,  32, 121, 155, 137, 114,  19,  99, 146,  11, 136, 220, 195,
       105, 170, 219,  98, 203, 217, 173,  64, 228, 127, 212,  42, 156,
       192, 182,  46,  59,  18, 102,  36, 168,  26,  87, 126, 12

In [10]:
def get_countries_population(country_ids, countries_metadata):
    """
    countries are ordered using country ids, we query for their population and return it accordingly.
    """
    return np.array(
        list(map(lambda cid: countries_metadata[str(cid)]["population"], country_ids))
    )

In [11]:
countries_population = get_countries_population(country_ids, countries_metadata)

In [12]:
incidences = (incidences / countries_population[np.newaxis, :, np.newaxis, np.newaxis])

In [13]:
print("min", incidences.min())
print("max", incidences.max())
print("mean", incidences.mean())
print("std", incidences.std())

min 0.0
max 0.04334470989761092
mean 0.0009119511804321677
std 0.0023550459485025665


Now that incidences is (hopefully) normalized by population, we can use sliding windows and create train test sets

In [15]:
with h5py.File('data/EpiGCN/processing/incidences_normalized.hdf5', 'w') as f:
    f.create_dataset('incidences', data=incidences, compression="lzf")

## Load incidences normalized

In [3]:
incidences_normalized = None
with h5py.File("data/EpiGCN/processing/incidences_normalized.hdf5", "r") as f:
    incidences_normalized = f['incidences'][:]

## Create sliding window dataset

In [4]:
incidences_normalized.shape

(3181, 232, 366, 4)

In [24]:
from IPython.display import clear_output

def generate_sliding_windows_dataset(incidences, t_lag=10, window_size=10):
    """
    :t_lag: 
    :window_size: assumes this is less than 366
    """
    with h5py.File("data/EpiGCN/dataset.hdf5", "w") as fw:
        x_ts = fw.create_dataset(
            "x_ts",
            compression="lzf",
            shape=((366 - t_lag - window_size + 1) * 3181, 232, window_size, 4),
            maxshape=(None, 232, window_size, 4),
        )
        y_ts = fw.create_dataset(
            "y_ts",
            compression="lzf",
            shape=((366 - t_lag - window_size + 1) * 3181, 232, 1, 4),
            maxshape=(None, 232, 1, 4),
        )

        start, end = t_lag + window_size, 367
        window_indices = np.arange(start, end, step=1)
        target_ix = 0
        for ix in list(reversed(window_indices)):
            print(ix, target_ix)
            
            x = incidences[:, :, (ix - window_size) : ix, :]
            y = incidences[:, :, (ix - window_size - t_lag), :]

            ## expand dims for y so that there is a single dimension for y
            ## and then swap last 2 axis so that counts are again the last axis
            y = np.swapaxes(np.expand_dims(y, -1), -1, -2)
            x_ts[target_ix * 3181: (target_ix + 1) * 3181] = x
            y_ts[target_ix * 3181: (target_ix + 1) * 3181] = y
            target_ix += 1
            clear_output(wait=True)
        #     return np.stack(x_data), np.stack(y_data)

In [25]:
generate_sliding_windows_dataset(incidences_normalized)

20 346


In [27]:
with h5py.File("data/EpiGCN/dataset.hdf5", "r") as f:
    print(f['x_ts'].shape)
    print(f['y_ts'].shape)

(1103807, 232, 10, 4)
(1103807, 232, 1, 4)


## Split into train and test

In [6]:
from sklearn.model_selection import train_test_split


def get_train_test_indices(t_lag=10, window_size=10, test_ratio=0.1, random_seed=42):
    num_samples = (366 - t_lag - window_size + 1) * 3181
    train_indices, test_indices = train_test_split(
        range(num_samples),
        test_size=int(num_samples * test_ratio),
        random_state=random_seed,
        shuffle=True,
    )
    return train_indices, test_indices


train_indices, test_indices = get_train_test_indices()

In [15]:
x_ts = None
y_ts = None

with h5py.File("data/EpiGCN/dataset.hdf5", "r") as f:
    x_ts = f["x_ts"][:]
    y_ts = f["y_ts"][:]

This loads the entire dataset in memory - approx 50 GB and splits into train test

In [16]:
def split_into_train_test_in_mem(train_indices, test_indices):
    x_train = x_ts[train_indices]
    y_train = y_ts[train_indices]
    x_test = x_ts[test_indices]
    y_test = y_ts[test_indices]
    
    print("finished splitting")
    with h5py.File('data/EpiGCN/processing/train_data.hdf5', 'w') as f:
        f.create_dataset('x_train', data=x_train, compression="lzf")
        f.create_dataset('y_train', data=y_train, compression="lzf")
        
    with h5py.File('data/EpiGCN/processing/test_data.hdf5', 'w') as f:
        f.create_dataset('x_test', data=x_test, compression="lzf")
        f.create_dataset('y_test', data=y_test, compression="lzf")

In [17]:
split_into_train_test_in_mem(train_indices, test_indices)

finished splitting


This method allows streaming train test split of the data

In [11]:
def split_into_train_test(train_incides, test_indices, window_size=10):
    with h5py.File("data/EpiGCN/dataset.hdf5", "r") as f:
        with h5py.File("data/EpiGCN/train_test.hdf5", "w") as fw:
            x_ts = f["x_ts"][:]
            print("x data loaded")

            y_ts = f["y_ts"][:]

            print("y data loaded")

            x_train = fw.create_dataset(
                "x_train",
                compression="lzf",
                shape=(len(train_indices), 232, window_size, 4),
                maxshape=(None, 232, window_size, 4),
            )
            y_train = fw.create_dataset(
                "y_train",
                compression="lzf",
                shape=(len(train_indices), 232, 1, 4),
                maxshape=(None, 232, 1, 4),
            )
            x_test = fw.create_dataset(
                "x_test",
                compression="lzf",
                shape=(len(test_indices), 232, window_size, 4),
                maxshape=(None, 232, window_size, 4),
            )
            y_test = fw.create_dataset(
                "y_test",
                compression="lzf",
                shape=(len(test_indices), 232, 1, 4),
                maxshape=(None, 232, 1, 4),
            )

            train_current_ix, test_current_ix = 0, 0
            batch_size = 5000
            for ix in range(0, len(train_indices), batch_size):
                print(
                    "Training progress: {:.2f}".format(
                        (train_current_ix * batch_size * 1.0) / len(train_incides)
                    )
                )
                ixes = sorted(train_indices[ix : ix + batch_size])
                x_train[ix : ix + batch_size] = x_ts[ixes]
                y_train[ix : ix + batch_size] = y_ts[ixes]
                train_current_ix += 1
                clear_output(wait=True)

            for test_ix in tqdm.tqdm_notebook(test_indices):
                print(
                    "Testing progress: {:.2f}".format(
                        (test_current_ix * 1.0) / len(test_indices)
                    )
                )
                x_test[test_current_ix] = x_ts[test_ix]
                y_test[test_current_ix] = y_ts[test_ix]
                test_current_ix += 1
                clear_output(wait=True)

In [18]:
# split_into_train_test(train_indices, test_indices)

In [2]:
x_train = None
y_train = None
x_test = None
y_test = None

with h5py.File('data/EpiGCN/processing/train_data.hdf5', 'r') as f:
    x_train = f['x_train'][:] 
    y_train = f['y_train'][:]

with h5py.File('data/EpiGCN/processing/test_data.hdf5', 'r') as f:
    x_test = f['x_test'][:] 
    y_test = f['y_test'][:]

In [13]:
print("y test size", y_test.shape)

y test size (110380, 232, 1, 4)


# Define Dataset

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset

train_dataset = TensorDataset(torch.FloatTensor(x_train), torch.FloatTensor(y_train))
test_dataset = TensorDataset(torch.FloatTensor(x_test), torch.FloatTensor(y_test))

INFO:wandb.run_manager:shutting down system stats and metadata service
INFO:wandb.run_manager:stopping streaming files and file change observer


# Define Dataloader

In [21]:
# params = {"batch_size": 12, "shuffle": True, "num_workers": 4}

train_dataloader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=12)
test_dataloader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=12)

## Model definition

In [17]:
import torch
import torch.nn as nn

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
class BaselineLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(BaselineLSTM, self).__init__()
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=True,
            num_layers=num_layers,
        )
        self.linear = nn.Linear(hidden_size * num_layers, input_size)

    def forward(self, lstm_input):
        b, n_countries, seq_len, n_compartments = lstm_input.size()
        lstm_input = lstm_input.permute(0, 2, 1, 3)
        _, (hn, _) = self.lstm(lstm_input.contiguous().view(b, seq_len, -1))
        hn = hn.permute(1, 0, 2)
        return self.linear(hn.reshape(b, -1)).view(b, n_countries, 1, n_compartments)


class BaselineGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(BaselineGRU, self).__init__()
        self.num_layers = num_layers
        self.lstm = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=True,
            num_layers=num_layers,
        )
        self.linear = nn.Linear(hidden_size * num_layers, input_size)

    def forward(self, lstm_input):
        b, n_countries, seq_len, n_compartments = lstm_input.size()
        lstm_input = lstm_input.permute(0, 2, 1, 3)
        _, hn = self.lstm(lstm_input.contiguous().view(b, seq_len, -1))
        hn = hn.permute(1, 0, 2)
        return self.linear(hn.reshape(b, -1)).view(b, n_countries, 1, n_compartments)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.handlers import ModelCheckpoint
from ignite.metrics import Loss
from ignite.contrib.handlers.tensorboard_logger import *
import wandb


class ModelTrainer:
    def __init__(self, run_name, **training_params):
        self.run_name = run_name
        self.training_params = training_params

        params_str = ",".join(
            ["{}={}".format(k, v) for k, v in training_params.items()]
        )
        wandb.init(sync_tensorboard=True, name=self.run_name)
        wandb.config.initial_lr = training_params["lr"]
        wandb.config.input_size = training_params["input_size"]
        wandb.config.hidden_size = training_params["hidden_size"]
        wandb.config.num_layers = training_params["num_layers"]
        wandb.config.param_str = params_str

    def train(self, model_class, max_epochs=100):
        model = model_class(
            input_size=self.training_params["input_size"],
            hidden_size=self.training_params["hidden_size"],
            num_layers=self.training_params["num_layers"],
        )

        wandb.watch(model)

        optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, patience=3, factor=0.5
        )

        mse = torch.nn.MSELoss()
        mae = torch.nn.L1Loss()

        tb_logger = TensorboardLogger(
            log_dir="experiments/tb_logs/runs/IID_all_compartments_{}".format(
                params_str
            )
        )

        trainer = create_supervised_trainer(model, optimizer, mae, device=device)

        metrics = {"mae": Loss(mae), "mse": Loss(mse)}

        train_evaluator = create_supervised_evaluator(
            model, metrics=metrics, device=device
        )
        validation_evaluator = create_supervised_evaluator(
            model, metrics=metrics, device=device
        )

        def score_function(trainer):
            validation_evaluator.run(test_dataloader)
            metrics = validation_evaluator.state.metrics
            mae = metrics["mae"]
            return -mae

        checkpointer = ModelCheckpoint(
            "model_checkpoints",
            "IID_all_compartments-" + params_str,
            score_function=score_function,
            score_name="mae",
            n_saved=2,
            create_dir=True,
            save_as_state_dict=True,
            require_empty=False,
        )

        trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpointer, {"epi": model})

        tb_logger.attach(
            trainer,
            log_handler=OutputHandler(
                tag="training", output_transform=lambda loss: {"loss": loss}
            ),
            event_name=Events.ITERATION_COMPLETED,
        )

        tb_logger.attach(
            train_evaluator,
            log_handler=OutputHandler(
                tag="training",
                metric_names=["mae", "mse"],
                global_step_transform=global_step_from_engine(trainer),
            ),
            event_name=Events.EPOCH_COMPLETED,
        )

        tb_logger.attach(
            validation_evaluator,
            log_handler=OutputHandler(
                tag="validation",
                metric_names=["mae", "mse"],
                global_step_transform=global_step_from_engine(trainer),
            ),
            event_name=Events.EPOCH_COMPLETED,
        )

        # Attach the logger to the trainer to log optimizer's parameters, e.g. learning rate at each iteration
        tb_logger.attach(
            trainer,
            log_handler=OptimizerParamsHandler(optimizer),
            event_name=Events.ITERATION_STARTED,
        )

        # Attach the logger to the trainer to log model's weights norm after each iteration
        tb_logger.attach(
            trainer,
            log_handler=WeightsScalarHandler(model),
            event_name=Events.ITERATION_COMPLETED,
        )

        # Attach the logger to the trainer to log model's weights as a histogram after each epoch
        tb_logger.attach(
            trainer,
            log_handler=WeightsHistHandler(model),
            event_name=Events.EPOCH_COMPLETED,
        )

        # Attach the logger to the trainer to log model's gradients norm after each iteration
        tb_logger.attach(
            trainer,
            log_handler=GradsScalarHandler(model),
            event_name=Events.ITERATION_COMPLETED,
        )

        # Attach the logger to the trainer to log model's gradients as a histogram after each epoch
        tb_logger.attach(
            trainer,
            log_handler=GradsHistHandler(model),
            event_name=Events.EPOCH_COMPLETED,
        )

        @trainer.on(Events.ITERATION_COMPLETED)
        def log_training_loss(trainer):
            logger.info(
                "Epoch[{}] Loss: {:.7f}".format(
                    trainer.state.epoch, trainer.state.output
                )
            )
            wandb.log({"train loss": trainer.state.output})

        @trainer.on(Events.EPOCH_COMPLETED)
        def log_training_results(trainer):
            train_evaluator.run(train_dataloader)
            metrics = train_evaluator.state.metrics
            logger.info(
                "Training Results - Epoch: {} MSE: {:.7f} MAE: {:.7f}".format(
                    trainer.state.epoch, metrics["mse"], metrics["mae"]
                )
            )

        @trainer.on(Events.EPOCH_COMPLETED)
        def log_validation_results(trainer):
            validation_evaluator.run(test_dataloader)
            metrics = validation_evaluator.state.metrics
            logger.info(
                "Validation Results - Epoch: {} MSE: {:.7f} MAE: {:.7f}".format(
                    trainer.state.epoch, metrics["mse"], metrics["mae"]
                )
            )
            lr_scheduler.step(metrics["mae"])
            wandb.log({"val mae": metrics["mae"]})
            wandb.log({"val mse": metrics["mse"]})

        trainer.run(train_dataloader, max_epochs=max_epochs)
        tb_logger.close()

In [ ]:
trainer = ModelTrainer('IID_all_compartments_GRU', input_size=232*4, hidden_size=200, num_layers=1, lr=1e-4)
trainer.train(BaselineGRU)

In [ ]:
# from torch.utils.tensorboard import SummaryWriter
# from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
# from ignite.handlers import ModelCheckpoint
# from ignite.metrics import Loss
# from ignite.contrib.handlers.tensorboard_logger import *
# import wandb
# wandb.init(sync_tensorboard=True, name='IID_all_compartments')

# params = {"lr": 0.0001, "input_size": 232 * 4, "hidden_size": 200, "num_layers": 1}

# params_str = ",".join(["{}={}".format(k, v) for k, v in params.items()])

# wandb.config.initial_lr = params['lr']
# wandb.config.input_size = params['input_size']
# wandb.config.hidden_size = params['hidden_size']
# wandb.config.num_layers = params['num_layers']
# wandb.config.param_str = params_str

# model = BaselineLSTM(
#     input_size=params["input_size"],
#     hidden_size=params["hidden_size"],
#     num_layers=params["num_layers"],
# )

# wandb.watch(model)

# optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])
# lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)

# mse = torch.nn.MSELoss()
# mae = torch.nn.L1Loss()

# tb_logger = TensorboardLogger(
#     log_dir="experiments/tb_logs/runs/IID_all_compartments_{}".format(params_str)
# )

# trainer = create_supervised_trainer(model, optimizer, mae, device=device)

# metrics = {"mae": Loss(mae), "mse": Loss(mse)}

# train_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)
# validation_evaluator = create_supervised_evaluator(
#     model, metrics=metrics, device=device
# )


# def score_function(trainer):
#     validation_evaluator.run(test_dataloader)
#     metrics = validation_evaluator.state.metrics
#     mae = metrics["mae"]
#     return -mae


# checkpointer = ModelCheckpoint(
#     "model_checkpoints",
#     "IID_all_compartments-" + params_str,
#     score_function=score_function,
#     score_name="mae",
#     n_saved=2,
#     create_dir=True,
#     save_as_state_dict=True,
#     require_empty=False,
# )

# trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpointer, {"epi": model})

# tb_logger.attach(
#     trainer,
#     log_handler=OutputHandler(
#         tag="training", output_transform=lambda loss: {"loss": loss}
#     ),
#     event_name=Events.ITERATION_COMPLETED,
# )

# tb_logger.attach(
#     train_evaluator,
#     log_handler=OutputHandler(
#         tag="training",
#         metric_names=["mae", "mse"],
#         global_step_transform=global_step_from_engine(trainer),
#     ),
#     event_name=Events.EPOCH_COMPLETED,
# )

# tb_logger.attach(
#     validation_evaluator,
#     log_handler=OutputHandler(
#         tag="validation",
#         metric_names=["mae", "mse"],
#         global_step_transform=global_step_from_engine(trainer),
#     ),
#     event_name=Events.EPOCH_COMPLETED,
# )

# # Attach the logger to the trainer to log optimizer's parameters, e.g. learning rate at each iteration
# tb_logger.attach(
#     trainer,
#     log_handler=OptimizerParamsHandler(optimizer),
#     event_name=Events.ITERATION_STARTED,
# )

# # Attach the logger to the trainer to log model's weights norm after each iteration
# tb_logger.attach(
#     trainer,
#     log_handler=WeightsScalarHandler(model),
#     event_name=Events.ITERATION_COMPLETED,
# )

# # Attach the logger to the trainer to log model's weights as a histogram after each epoch
# tb_logger.attach(
#     trainer, log_handler=WeightsHistHandler(model), event_name=Events.EPOCH_COMPLETED
# )

# # Attach the logger to the trainer to log model's gradients norm after each iteration
# tb_logger.attach(
#     trainer,
#     log_handler=GradsScalarHandler(model),
#     event_name=Events.ITERATION_COMPLETED,
# )

# # Attach the logger to the trainer to log model's gradients as a histogram after each epoch
# tb_logger.attach(
#     trainer, log_handler=GradsHistHandler(model), event_name=Events.EPOCH_COMPLETED
# )


# @trainer.on(Events.ITERATION_COMPLETED)
# def log_training_loss(trainer):
#     logger.info(
#         "Epoch[{}] Loss: {:.7f}".format(trainer.state.epoch, trainer.state.output)
#     )
#     wandb.log({"train loss": trainer.state.output})


# @trainer.on(Events.EPOCH_COMPLETED)
# def log_training_results(trainer):
#     train_evaluator.run(train_dataloader)
#     metrics = train_evaluator.state.metrics
#     logger.info(
#         "Training Results - Epoch: {} MSE: {:.7f} MAE: {:.7f}".format(
#             trainer.state.epoch, metrics["mse"], metrics["mae"]
#         )
#     )


# @trainer.on(Events.EPOCH_COMPLETED)
# def log_validation_results(trainer):
#     validation_evaluator.run(test_dataloader)
#     metrics = validation_evaluator.state.metrics
#     logger.info(
#         "Validation Results - Epoch: {} MSE: {:.7f} MAE: {:.7f}".format(
#             trainer.state.epoch, metrics["mse"], metrics["mae"]
#         )
#     )
#     lr_scheduler.step(metrics["mae"])
#     wandb.log({"val mae": metrics['mae']})
#     wandb.log({"val mse": metrics['mse']})


# trainer.run(train_dataloader, max_epochs=100)
# # We need to close the logger with we are done
# tb_logger.close()

ERROR:wandb.jupyter:Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: WARNING Not logging key "IID_all_compartments_lr=0.0001,input_size=928,hidden_size=200,num_layers=1/weights/lstm/weight_ih_l0".  Histograms must have fewer than 512 bins
wandb: WARNING Not logging key "IID_all_compartments_lr=0.0001,input_size=928,hidden_size=200,num_layers=1/weights/lstm/weight_hh_l0".  Histograms must have fewer than 512 bins
wandb: WARNING Not logging key "IID_all_compartments_lr=0.0001,input_size=928,hidden_size=200,num_layers=1/weights/lstm/bias_ih_l0".  Histograms must have fewer than 512 bins
wandb: WARNING Not logging key "IID_all_compartments_lr=0.0001,input_size=928,hidden_size=200,num_layers=1/weights/lstm/bias_hh_l0".  Histograms must have fewer than 512 bins
wandb: WARNING Not logging key "IID_all_compartments_lr=0.0001,input_size=928,hidden_size=200,num_layers=1/weights/linear/weight".  Histograms must have fewer than 512